In [16]:
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnableMap
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import LLMChain

In [17]:
# Load and split documents
loader = TextLoader("../files/chapter_three.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

# Initialize embeddings and vectorstore
cache_dir = LocalFileStore("./.cache/")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# Define the LLM and retriever
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.1)
try:
    retriever = vectorstore.as_retriever()
except AttributeError as e:
    raise RuntimeError("Failed to initialize retriever from vectorstore. Ensure the vectorstore is properly set up.") from e


In [18]:
# Define a custom prompt template for the StuffDocuments chain
prompt_template = PromptTemplate(
    template="""
    Context:
    {context}

    Question:
    {question}

    Answer:
    """,
    input_variables=["context", "question"]
)

In [19]:
# Initialize memory for conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [20]:
# Create the StuffDocuments chain manually
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

In [21]:
# Create the full pipeline using RunnableMap
from langchain.schema.runnable import RunnablePassthrough
pipeline = RunnableMap({
    "retriever": retriever,
    "memory": RunnablePassthrough(memory),
    "qa_chain": stuff_chain
})

ValidationError: 2 validation errors for RunnablePassthrough
func.callable
  Input should be callable [type=callable_type, input_value=ConversationBufferMemory(...mory_key='chat_history'), input_type=ConversationBufferMemory]
    For further information visit https://errors.pydantic.dev/2.9/v/callable_type
func.callable
  Input should be callable [type=callable_type, input_value=ConversationBufferMemory(...mory_key='chat_history'), input_type=ConversationBufferMemory]
    For further information visit https://errors.pydantic.dev/2.9/v/callable_type